In [1]:
import os
import csv
import numpy as np

In [2]:
main_dir ="Results_union"
old_drifts = ["sea_P(y)","sea_P(y_x)","sea_P(x_y)","sine_P(y)","sine_P(y_x)","sine_P(x_y)"]
new_drifts = ["jitter","disappearing-minority","appearing-minority","appearing-clusters",
             "shapeshift","minority-share","splitting-clusters","clusters-movement","borderline"]
type_drift = ["incremental","sudden","periodic"]
unbalance_lvls = [1,2,3,4]
stats_w = ["Recall_0","Recall_1", "F1_0", "F1_1", "Gmean"]
stats_tot =["time", "memory"]
algs = ["RebalanceStream","CSMOTE","improved_UOB","improved_OOB","WEOB1","WEOB2","ESOS_ELM","UOB","OOB"]

In [3]:
def import_csv(csvfilename):
    data = []
    with open(csvfilename, "r") as scraped:
        reader = csv.reader(scraped, delimiter=',')
        row_index = 0
        for row in reader:
            if row:  # avoid blank lines
                row_index += 1
                columns = [str(row_index)]+ [row[i] for i in range(len(row))]
                data.append(columns)
        #print(f"{row_index} valid rows")
        
    return data

# MEAN ALL ROWS

# OLD

In [4]:
final_dict = {}

for old in old_drifts:
    path = main_dir + "/" + old[old.index("P"):]   #old = sea_P(y)
    if old not in final_dict:
        final_dict[old] = {}
        
    for ds in os.listdir(path):
        #check if ds in old (directory giusta)
        if old[:3] == ds[:3]:
            path1 = path + "/" + ds
            
            unbalance = 0
            for u in unbalance_lvls:
                if str(u) in ds:
                    unbalance = u
            assert unbalance != 0
                        
            if unbalance not in final_dict[old]:
                final_dict[old][unbalance] = {}
                
            dtype = "sudden"
            if "G" in ds:
                dtype = "incremental"
                
            if dtype not in final_dict[old][unbalance]:
                final_dict[old][unbalance][dtype] = {}

            for alg in os.listdir(path1):
                if "DS_Store" in alg:
                    continue
                path2 = path1 + "/" + alg
                class0 = []
                class1 = []
                
                print(f"{old} {unbalance} {dtype} {alg}")
                #prendo i valori di tutti i 10 esperimenti
                for result in os.listdir(path2):
                    if "err" not in result or "DS_Store" in result:
                        continue
                    start = False
                    data = import_csv(path2+"/"+ result)
                    daily0 = []
                    daily1 = []
                    for data_row in data:
                        if data_row[1] == "learning evaluation instances":
                            start = True
                            continue
                        if not start:
                            continue
                        
                        try:
                            daily0.append(float(data_row[9]))
                        except:
                            if data_row[9] == "?":
                                daily0.append(0)
                            else:
                                print("ZERO")
                                print(data_row)
                        try:
                            daily1.append(float(data_row[10]))
                        except:
                            if data_row[10] == "?":
                                daily1.append(0)
                            else:
                                print("ONE")
                                print(data_row)
                    if len(daily0)>0 and len(daily1)>0:
                        class0.append(np.mean(daily0))
                        class1.append(np.mean(daily1))
                            
                        
                    
                assert len(class0) > 0
                assert len(class1) > 0
                assert len(class1) == len(class0)
                if np.mean(class0) != np.mean(class0):
                    print(class0)
                if np.mean(class1) != np.mean(class1):
                    print(class1)
                if alg == "OSHAT":
                    alg = "CSMOTE"
                final_dict[old][unbalance][dtype][alg] = {}
                final_dict[old][unbalance][dtype][alg]["class0"] = np.mean(class0)
                final_dict[old][unbalance][dtype][alg]["class1"] = np.mean(class1)

                print(np.mean(class0))
                print(np.mean(class1))
                    
                    
                
        

sea_P(y) 1 sudden improved_OOB
99.04130893811295
98.10819729063016
sea_P(y) 1 sudden WEOB1
98.97942226181465
97.83921843449522
sea_P(y) 1 sudden OOB
98.98612670033285
98.03087014819405
sea_P(y) 1 sudden ESOS_ELM
89.30415619559611
73.42351874845741
sea_P(y) 1 sudden improved_UOB
98.81573147725297
96.9240271514656
sea_P(y) 1 sudden WEOB2
98.6244127997512
96.19870135316566
sea_P(y) 1 sudden UOB
98.98612670033285
98.03087014819405
sea_P(y) 1 sudden OSHAT
98.74740154470427
97.13040882292873
sea_P(y) 1 sudden RebalanceStream
98.4522316004296
93.46004270685152
sea_P(y) 3 incremental improved_OOB
88.08041188023472
80.58624789587222
sea_P(y) 3 incremental WEOB1
88.0473140329549
80.63956462559813
sea_P(y) 3 incremental OOB
87.78576702893473
80.36644102090415
sea_P(y) 3 incremental ESOS_ELM
82.41503576802805
63.45563049312749
sea_P(y) 3 incremental improved_UOB
87.73204260284595
80.2582474829982
sea_P(y) 3 incremental WEOB2
87.13733641570286
79.81299251202663
sea_P(y) 3 incremental UOB
87.6816893

85.12746657850657
60.12601746519365
sea_P(y_x) 2 incremental OSHAT
87.46231358011173
60.99464766018574
sea_P(y_x) 2 incremental RebalanceStream
90.91032302421948
51.31017005743174
sea_P(y_x) 4 sudden improved_OOB
84.98443973509949
78.38035652617066
sea_P(y_x) 4 sudden WEOB1
84.88722679721056
78.22086326320559
sea_P(y_x) 4 sudden OOB
84.77730113892997
79.2364676934806
sea_P(y_x) 4 sudden ESOS_ELM
81.21121484849324
61.83718447652819
sea_P(y_x) 4 sudden improved_UOB
84.419685975693
77.59199342741667
sea_P(y_x) 4 sudden WEOB2
82.4376348103982
69.42351001442499
sea_P(y_x) 4 sudden UOB
83.32791008804074
77.79005542537746
sea_P(y_x) 4 sudden OSHAT
84.36629003563084
77.88115345992912
sea_P(y_x) 4 sudden RebalanceStream
84.26079458473279
77.05709044101523
sea_P(x_y) 1 sudden improved_OOB
94.72620198344217
62.759735047197715
sea_P(x_y) 1 sudden WEOB1
92.79533132832
57.414892977741054
sea_P(x_y) 1 sudden OOB
93.93195376474775
60.19540049911734
sea_P(x_y) 1 sudden ESOS_ELM
92.71794955463612
44.691

98.3511321138681
96.38275111904463
sine_P(y) 2 incremental UOB
98.55683959548938
96.69352921941831
sine_P(y) 2 incremental OSHAT
97.637939598043
96.67757574789115
sine_P(y) 2 incremental RebalanceStream
97.06340334882798
95.60598947045503
sine_P(y) 1 incremental improved_OOB
99.03347540628508
97.6354689746473
sine_P(y) 1 incremental WEOB1
99.02810240573226
97.34930857831986
sine_P(y) 1 incremental OOB
99.19984614290983
97.09395387623707
sine_P(y) 1 incremental ESOS_ELM
94.80152454378573
77.69834450430776
sine_P(y) 1 incremental improved_UOB
97.9835978128111
92.73186633575952
sine_P(y) 1 incremental WEOB2
97.45531083584845
92.54726847542864
sine_P(y) 1 incremental UOB
98.40462686350568
94.58036248321693
sine_P(y) 1 incremental OSHAT
96.6306832781323
91.93287610734868
sine_P(y) 1 incremental RebalanceStream
97.21222693364803
93.57971730157962
sine_P(y) 2 sudden improved_OOB
99.20618927651536
98.35665889809317
sine_P(y) 2 sudden WEOB1
99.17844201478128
98.02574459550907
sine_P(y) 2 sudden

99.2322414975753
98.86737831768696
sine_P(x_y) 4 sudden OOB
99.35443629525426
99.03234628120268
sine_P(x_y) 4 sudden ESOS_ELM
92.085486527406
78.93582104822516
sine_P(x_y) 4 sudden improved_UOB
98.96277152634195
98.48284670037606
sine_P(x_y) 4 sudden WEOB2
98.91653346501666
98.41166868844925
sine_P(x_y) 4 sudden UOB
99.35443629525426
99.03234628120268
sine_P(x_y) 4 sudden OSHAT
98.85466027668578
98.3217171763931
sine_P(x_y) 4 sudden RebalanceStream
98.62096377859567
97.9880061132042
sine_P(x_y) 1 sudden improved_OOB
99.6780866429249
97.02958144197942
sine_P(x_y) 1 sudden WEOB1
99.6176783933378
96.53116035601798
sine_P(x_y) 1 sudden OOB
99.60403578704776
96.41061487741982
sine_P(x_y) 1 sudden ESOS_ELM
97.47874092398634
70.662466422781
sine_P(x_y) 1 sudden improved_UOB
98.98729770780798
91.45307074556843
sine_P(x_y) 1 sudden WEOB2
98.47318718659346
87.6849496163838
sine_P(x_y) 1 sudden UOB
98.56721005157252
88.41909690260096
sine_P(x_y) 1 sudden OSHAT
97.96231598040663
83.23099619051092


In [7]:
old = final_dict.copy()
old

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'class0': 99.04130893811295,
     'class1': 98.10819729063016},
    'WEOB1': {'class0': 98.97942226181465, 'class1': 97.83921843449522},
    'OOB': {'class0': 98.98612670033285, 'class1': 98.03087014819405},
    'ESOS_ELM': {'class0': 89.30415619559611, 'class1': 73.42351874845741},
    'improved_UOB': {'class0': 98.81573147725297, 'class1': 96.9240271514656},
    'WEOB2': {'class0': 98.6244127997512, 'class1': 96.19870135316566},
    'UOB': {'class0': 98.98612670033285, 'class1': 98.03087014819405},
    'CSMOTE': {'class0': 98.74740154470427, 'class1': 97.13040882292873},
    'RebalanceStream': {'class0': 98.4522316004296,
     'class1': 93.46004270685152}},
   'incremental': {'improved_OOB': {'class0': 99.07781073529245,
     'class1': 98.36413751297053},
    'WEOB1': {'class0': 99.00395733265307, 'class1': 98.04731708900161},
    'OOB': {'class0': 99.0202469306703, 'class1': 98.31815573804992},
    'ESOS_ELM': {'class0': 89.7427966771706

# NEW

In [8]:
final_dict = old.copy()

In [9]:
for new in new_drifts:
    path = main_dir + "/" + new 
    
    if new not in final_dict:
        final_dict[new] = {}
        
    for unbalance in range(1,5):
        if unbalance not in final_dict[new]:
            final_dict[new][unbalance] = {}


        path1 = path + "/" + str(unbalance)

        for alg in os.listdir(path1):
            if "DS_Store" in alg:
                continue
            path2 = path1 + "/" + alg
            
            alg_dtypes = {}
            alg_dtypes["incremental"] = {}
            alg_dtypes["sudden"] = {}
            alg_dtypes["periodic"] = {}
            alg_dtypes["incremental"]["class0"] = []
            alg_dtypes["sudden"]["class0"] = []
            alg_dtypes["periodic"]["class0"] = []
            alg_dtypes["incremental"]["class1"] = []
            alg_dtypes["sudden"]["class1"] = []
            alg_dtypes["periodic"]["class1"] = []
            
            
            print(f"{new} {unbalance} {alg}")
            #prendo i valori di tutti i 10 esperimenti dei 3 dtype
            
            for result in os.listdir(path2):
                if "err" not in result or "DS_Store" in result:
                    continue
                daily0 = []
                daily1 = []
                data = import_csv(path2+"/"+ result)
                start = False
                
                for data_row in data:
                    if data_row[1] == "learning evaluation instances":
                        start = True
                        continue
                    if not start:
                        continue

                    try:
                        daily0.append(float(data_row[9]))
                    except:
                        if data_row[9] == "?":
                            daily0.append(0.0)
                        else:
                            print("ZERO")
                            print(data_row)
                    try:
                        daily1.append(float(data_row[10]))
                    except:
                        if data_row[10] == "?":
                            daily1.append(0.0)
                        else:
                            print("ONE")
                            print(data_row)
                        
                    
                
                try:
                    if len(daily0)>0 and len(daily1)>0:
                        if alg == "ESOS_ELM" and new == "appearing-minority":
                            print(daily0)
                            print(daily1)
                        alg_dtypes[result[:result.index("_")]]["class0"].append(np.mean(daily0))
                        alg_dtypes[result[:result.index("_")]]["class1"].append(np.mean(daily1))
                except:
                    print(f"error with {result}")
            for k,i in alg_dtypes.items():      
                if len(i["class0"]) < 10:
                    print(f"N EXP {len(i['class0'])} in 0")
                if len(i["class1"]) < 10:
                    print(f"N EXP {len(i['class1'])} in 1")
                if k not in final_dict[new][unbalance]:
                    final_dict[new][unbalance][k] = {}
                assert alg not in final_dict[new][unbalance][k]
                final_dict[new][unbalance][k][alg] = {}
                final_dict[new][unbalance][k][alg]["class0"] = np.mean(i["class0"])
                final_dict[new][unbalance][k][alg]["class1"] = np.mean(i["class1"])
                print(k)
                print(np.mean(i["class1"]))
                print(np.mean(i["class0"]))

jitter 1 improved_OOB
incremental
57.37133226312047
92.56181751723093
sudden
75.55851762348355
95.70163543712847
periodic
57.37133226312047
92.56181751723093
jitter 1 WEOB1
incremental
53.6243098407444
90.4801850117582
sudden
72.8682131895415
94.70571906938414
periodic
53.6243098407444
90.4801850117582
jitter 1 CSMOTE
incremental
42.77060637417368
86.0907328276071
sudden
47.74465185542761
88.01308046367015
periodic
42.77060637417368
86.0907328276071
jitter 1 OOB
incremental
54.83374234342974
91.18250060393169
sudden
73.58168785955442
94.725496172701
periodic
54.83374234342974
91.18250060393169
jitter 1 ESOS_ELM
incremental
34.1052838404195
84.46844793787247
sudden
55.50938218131704
93.19340712062409
periodic
33.59323904616951
84.2229182286193
jitter 1 improved_UOB
incremental
44.149803923449596
85.00700687090594
sudden
55.52449705489572
88.76459014182113
periodic
44.149803923449596
85.00700687090594
jitter 1 WEOB2
incremental
43.1823170557803
84.25716985850703
sudden
47.218436957326006

incremental
29.603769999651366
94.99728926924203
sudden
27.145230107854672
95.03974162097549
periodic
45.4977157788869
90.31318859029635
disappearing-minority 3 improved_OOB
incremental
42.10546308242948
93.09964454802345
sudden
39.17834254608319
93.23295142249117
periodic
78.53919566143067
88.90972701044416
disappearing-minority 3 WEOB1
incremental
41.61808282242059
92.2959620938453
sudden
38.68829011005202
92.6661720045237
periodic
78.39361160656044
88.47779973321963
disappearing-minority 3 CSMOTE
incremental
41.05504253089475
92.53990786068297
sudden
38.63563806683284
92.81597442277179
periodic
76.68147001187914
87.67874797845482
disappearing-minority 3 OOB
incremental
41.921994450520046
92.65073033911958
sudden
38.95956029436961
92.7679407825923
periodic
77.88794424634645
88.01724783329746
disappearing-minority 3 ESOS_ELM
incremental
25.60951158194164
89.72446376154852
sudden
23.094213956494094
90.60992044781736
periodic
57.1376436620748
82.39269151501252
disappearing-minority 3 im

incremental
20.870916595690154
91.02141293232448
sudden
20.387979433057204
91.63203869590981
periodic
2.907002713677648
91.30304381190082
appearing-minority 1 WEOB2
incremental
20.986663682372658
91.46914315414361
sudden
20.36925877608548
91.87719393282168
periodic
2.905459175553079
91.34997867578954
appearing-minority 1 UOB
incremental
27.5684819948494
93.88589202160486
sudden
19.613837016330343
91.27245771444883
periodic
3.4980972302999334
94.2836549320687
appearing-minority 1 RebalanceStream
incremental
17.952599965956832
97.80116204367695
sudden
6.7606396410278435
97.52445926917281
periodic
5.476014480392715
99.78421470325735
appearing-minority 2 improved_OOB
incremental
40.552037843722026
95.78117341056686
sudden
35.91913150499728
94.99733646277512
periodic
7.459649129496458
99.61289734506194
appearing-minority 2 WEOB1
incremental
38.602102011746695
94.80259451832714
sudden
35.0021959731456
94.32978767506424
periodic
6.616451898083419
99.05445865245784
appearing-minority 2 CSMOTE


incremental
32.2497494474593
91.21283720833596
sudden
31.009069828271617
91.57692310817406
periodic
4.581469755599109
88.71130052543414
appearing-minority 2 WEOB2
incremental
31.31249480356949
90.36790582072652
sudden
30.22372774078036
90.93550785178681
periodic
4.648546413067324
89.11706206271876
appearing-minority 2 UOB
incremental
37.427545117165415
94.67494645931245
sudden
31.307699244637824
91.89165154408035
periodic
4.988729558374645
91.30179897549965
appearing-minority 2 RebalanceStream
incremental
23.76954859834754
94.92503320991759
sudden
24.893118953751234
94.82680618770556
periodic
6.346650173332722
99.62690123185772
appearing-minority 3 improved_OOB
incremental
44.25452415743781
95.14594117437835
sudden
40.309301626171326
94.60737227374342
periodic
7.5057708509149395
99.37303707978899
appearing-minority 3 WEOB1
incremental
42.3558178485214
94.25823500435949
sudden
39.8621697866793
94.09339175639357
periodic
6.805785779148709
98.56653290230045
appearing-minority 3 CSMOTE
inc

incremental
39.89620814398902
92.50078730833863
sudden
37.21357885308089
91.7964352120459
periodic
5.59734109919504
89.50992518453117
appearing-minority 3 WEOB2
incremental
38.72348833727115
91.34828689748629
sudden
36.562196322024015
91.24810351792681
periodic
5.641779603011637
91.57451818381745
appearing-minority 3 UOB
incremental
40.85092551539442
93.32011262713196
sudden
37.66510583595267
92.16050869069032
periodic
6.110582437388926
91.44024264562483
appearing-minority 3 RebalanceStream
incremental
38.44119758802783
93.79720563419937
sudden
35.89590153618135
93.34841517626697
periodic
5.962475380781001
99.13472886963974
appearing-minority 4 improved_OOB
incremental
47.81243188898928
95.00515606013448
sudden
42.96168483136866
94.09666120093816
periodic
7.824024818506797
99.29770699182733
appearing-minority 4 WEOB1
incremental
46.76995578451033
94.42242044263989
sudden
42.633579062978086
93.6231275031931
periodic
7.250671267153879
98.70008064628692
appearing-minority 4 CSMOTE
increme

incremental
44.557424700251836
94.1734140145011
sudden
40.315422435743116
92.28537889288354
periodic
5.510022926463913
98.90645199797828
appearing-clusters 1 improved_OOB
incremental
65.90728652539545
94.31667876181311
sudden
66.89280377485144
94.5448411974393
periodic
76.2618549566175
96.55152299625188
appearing-clusters 1 WEOB1
incremental
64.60346190666675
93.80237423128128
sudden
64.61170712052261
93.77527150966282
periodic
74.24920625580289
96.09922740985385
appearing-clusters 1 CSMOTE
incremental
65.70502492757362
94.98153743256016
sudden
66.6381136936992
95.23742120310371
periodic
75.21938346241538
96.53459870548633
appearing-clusters 1 OOB
incremental
66.7160010795512
94.46259420256693
sudden
65.2259235861293
93.89588789921017
periodic
75.76469090011823
96.4392046771861
appearing-clusters 1 ESOS_ELM
incremental
36.69139477848908
86.69801492755002
sudden
36.56662908882637
86.7317946723961
periodic
40.58515853354128
88.5761424765391
appearing-clusters 1 improved_UOB
incremental
6

incremental
65.78965111723656
85.86681171839845
sudden
65.55362716829003
85.74086003486266
periodic
66.14244686331128
85.97123763975695
shapeshift 2 WEOB2
incremental
64.63459337649371
85.24481665741823
sudden
64.29422629900628
85.08188911342036
periodic
65.51225160983383
85.69981133883809
shapeshift 2 UOB
incremental
64.10863717677049
84.32691593403777
sudden
64.00233832213101
84.27574237186447
periodic
64.9895508287652
84.83109247261483
shapeshift 2 RebalanceStream
incremental
62.03818454615767
91.60289702933599
sudden
61.798291226747175
91.50174639388945
periodic
63.23711791429821
91.80965388103752
shapeshift 3 improved_OOB
incremental
79.45214382539179
88.31163420404354
sudden
79.55875335181767
88.47741558266544
periodic
79.21176476022222
88.05317544598395
shapeshift 3 WEOB1
incremental
78.79612707090124
87.70001182332994
sudden
78.83839354397074
87.77015824859163
periodic
78.55455450917239
87.41906691625817
shapeshift 3 CSMOTE
incremental
76.91484266792258
86.55243194323714
sudden

incremental
81.91555883209044
86.43277290087948
sudden
84.26258064082631
86.20367881197281
periodic
43.339517611478016
86.38495257760073
minority-share 4 UOB
incremental
82.61533278817798
87.61342484424975
sudden
85.96991762747577
88.63499472758096
periodic
44.463342713774914
87.77049249668809
minority-share 4 RebalanceStream
incremental
74.53400541880629
85.83169850989228
sudden
84.68496689618465
87.77967308439632
periodic
40.6284516665732
92.4902283908215
splitting-clusters 1 improved_OOB
incremental
64.60510566856018
94.1906090132541
sudden
64.09691908984142
94.00235940939294
periodic
72.04953090260041
95.90385250569666
splitting-clusters 1 WEOB1
incremental
63.335791914034566
93.44370500038816
sudden
62.132728237819244
93.24381798460169
periodic
69.97745851359063
95.32658515576465
splitting-clusters 1 CSMOTE
incremental
56.79142694082691
92.72139034783258
sudden
58.27268311278499
93.31530805539467
periodic
66.08020107480377
94.689816454681
splitting-clusters 1 OOB
incremental
63.92

55.71411189857516
80.75197992001657
sudden
52.792958612845595
82.25404663222812
periodic
64.64771841414313
86.22763990122344
clusters-movement 2 OOB
incremental
69.81448131201651
88.534117517812
sudden
70.03366857546163
88.65795726973703
periodic
69.01618231492714
88.09987808963942
clusters-movement 2 ESOS_ELM
incremental
50.90405733779964
83.98734932242319
sudden
48.62417988798639
84.58536024137656
periodic
49.574369992705336
83.9347718636988
clusters-movement 2 improved_UOB
incremental
65.40537323336454
86.0205095947779
sudden
63.21340251420148
85.27192080717944
periodic
61.06993960849078
83.18594761670998
clusters-movement 2 WEOB2
incremental
59.20584515566063
83.64223038750171
sudden
54.67008288592952
82.39140223957114
periodic
62.61032841394577
84.38497774284772
clusters-movement 2 UOB
incremental
62.76234268028371
84.24005852215227
sudden
57.49072266523532
79.27972736000973
periodic
59.70565242554843
81.54175723393686
clusters-movement 2 RebalanceStream
incremental
58.88580688887

incremental
85.63239600655285
87.81715632618747
sudden
85.41266768899035
87.68472149131694
periodic
86.90866595914056
89.1551224548758
borderline 4 OOB
incremental
86.94399828836421
89.49191825263618
sudden
86.10489541054713
88.72362540796027
periodic
89.17351862009818
91.42580665473167
borderline 4 ESOS_ELM
incremental
65.34489814341147
80.18036466427235
sudden
65.09267791455942
80.25520033028944
periodic
66.30061948080993
81.4342989342725
borderline 4 improved_UOB
incremental
85.70161366877565
87.70537197896985
sudden
85.02324614950042
87.07491624714694
periodic
87.90161539781568
89.76087786330514
borderline 4 WEOB2
incremental
85.03040197018424
87.00784009142123
sudden
84.22550581676958
86.2198847405879
periodic
86.88417031594689
88.7753348547004
borderline 4 UOB
incremental
86.94399828836421
89.49191825263618
sudden
86.10489541054713
88.72362540796027
periodic
89.17351862009818
91.42580665473167
borderline 4 RebalanceStream
incremental
85.67723867951551
88.60958615689148
sudden
85.

In [10]:
final_dict

{'sea_P(y)': {1: {'sudden': {'improved_OOB': {'class0': 99.04130893811295,
     'class1': 98.10819729063016},
    'WEOB1': {'class0': 98.97942226181465, 'class1': 97.83921843449522},
    'OOB': {'class0': 98.98612670033285, 'class1': 98.03087014819405},
    'ESOS_ELM': {'class0': 89.30415619559611, 'class1': 73.42351874845741},
    'improved_UOB': {'class0': 98.81573147725297, 'class1': 96.9240271514656},
    'WEOB2': {'class0': 98.6244127997512, 'class1': 96.19870135316566},
    'UOB': {'class0': 98.98612670033285, 'class1': 98.03087014819405},
    'CSMOTE': {'class0': 98.74740154470427, 'class1': 97.13040882292873},
    'RebalanceStream': {'class0': 98.4522316004296,
     'class1': 93.46004270685152}},
   'incremental': {'improved_OOB': {'class0': 99.07781073529245,
     'class1': 98.36413751297053},
    'WEOB1': {'class0': 99.00395733265307, 'class1': 98.04731708900161},
    'OOB': {'class0': 99.0202469306703, 'class1': 98.31815573804992},
    'ESOS_ELM': {'class0': 89.7427966771706

In [4]:
def stats_of(_dict ):
    values_0 = []
    values_1 = []
    for alg, values in _dict.items():
        if "ELM" in alg:
            continue
        values_0.append(values["class0"])
        values_1.append(values["class1"])
    return np.mean(values_0),np.mean(values_1), np.max(values_0), np.max(values_1), np.min(values_0), np.min(values_1)


In [16]:

with open('F1XF1_NOELM_PERC.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["drift","unbalance","type","alg","F1_0", "F1_1"])
    for drift, dicts in final_dict.items():
        for unbalance, dtypes in dicts.items():
            for dtype, algs in dtypes.items():
                mean0, mean1,max0,max1,min0,min1 = stats_of(algs)
                for alg, values in algs.items():
                    if "ELM" in alg:
                         continue
                    print([drift,unbalance,dtype,alg])
                    print(values)
                    value_0 = (values["class0"]/mean0)
                    value_1 = (values["class1"]/mean1)
                    writer.writerow([drift,unbalance,dtype,alg,value_0,value_1])

['sea_P(y)', 1, 'sudden', 'improved_OOB']
{'class0': 99.04130893811295, 'class1': 98.10819729063016}
['sea_P(y)', 1, 'sudden', 'WEOB1']
{'class0': 98.97942226181465, 'class1': 97.83921843449522}
['sea_P(y)', 1, 'sudden', 'OOB']
{'class0': 98.98612670033285, 'class1': 98.03087014819405}
['sea_P(y)', 1, 'sudden', 'improved_UOB']
{'class0': 98.81573147725297, 'class1': 96.9240271514656}
['sea_P(y)', 1, 'sudden', 'WEOB2']
{'class0': 98.6244127997512, 'class1': 96.19870135316566}
['sea_P(y)', 1, 'sudden', 'UOB']
{'class0': 98.98612670033285, 'class1': 98.03087014819405}
['sea_P(y)', 1, 'sudden', 'CSMOTE']
{'class0': 98.74740154470427, 'class1': 97.13040882292873}
['sea_P(y)', 1, 'sudden', 'RebalanceStream']
{'class0': 98.4522316004296, 'class1': 93.46004270685152}
['sea_P(y)', 1, 'incremental', 'improved_OOB']
{'class0': 99.07781073529245, 'class1': 98.36413751297053}
['sea_P(y)', 1, 'incremental', 'WEOB1']
{'class0': 99.00395733265307, 'class1': 98.04731708900161}
['sea_P(y)', 1, 'incremen

['disappearing-minority', 2, 'periodic', 'RebalanceStream']
{'class0': 90.31318859029635, 'class1': 45.4977157788869}
['disappearing-minority', 3, 'incremental', 'improved_OOB']
{'class0': 93.09964454802345, 'class1': 42.10546308242948}
['disappearing-minority', 3, 'incremental', 'WEOB1']
{'class0': 92.2959620938453, 'class1': 41.61808282242059}
['disappearing-minority', 3, 'incremental', 'CSMOTE']
{'class0': 92.53990786068297, 'class1': 41.05504253089475}
['disappearing-minority', 3, 'incremental', 'OOB']
{'class0': 92.65073033911958, 'class1': 41.921994450520046}
['disappearing-minority', 3, 'incremental', 'improved_UOB']
{'class0': 85.4732415699423, 'class1': 40.34659079190571}
['disappearing-minority', 3, 'incremental', 'WEOB2']
{'class0': 85.43164470873606, 'class1': 39.72899594306513}
['disappearing-minority', 3, 'incremental', 'UOB']
{'class0': 83.80057284472048, 'class1': 39.06529361208989}
['disappearing-minority', 3, 'incremental', 'RebalanceStream']
{'class0': 93.41291620468

['minority-share', 3, 'periodic', 'CSMOTE']
{'class0': 92.24672329735242, 'class1': 39.936851096849544}
['minority-share', 3, 'periodic', 'OOB']
{'class0': 92.44993344405552, 'class1': 40.831828812879635}
['minority-share', 3, 'periodic', 'improved_UOB']
{'class0': 87.36824081949655, 'class1': 40.22190778199506}
['minority-share', 3, 'periodic', 'WEOB2']
{'class0': 86.55601733175175, 'class1': 39.46773140355431}
['minority-share', 3, 'periodic', 'UOB']
{'class0': 85.42239865817476, 'class1': 38.90828750612644}
['minority-share', 3, 'periodic', 'RebalanceStream']
{'class0': 93.4369549867499, 'class1': 34.06275045717762}
['minority-share', 4, 'incremental', 'improved_OOB']
{'class0': 87.9896133362233, 'class1': 82.92825587300237}
['minority-share', 4, 'incremental', 'WEOB1']
{'class0': 88.03146228157325, 'class1': 83.4879608826653}
['minority-share', 4, 'incremental', 'CSMOTE']
{'class0': 88.17849780905195, 'class1': 82.54291182446818}
['minority-share', 4, 'incremental', 'OOB']
{'class0

{'class0': 93.76210588033133, 'class1': 87.25536232095075}
['splitting-clusters', 3, 'periodic', 'OOB']
{'class0': 94.469273235419, 'class1': 88.41758218628162}
['splitting-clusters', 3, 'periodic', 'improved_UOB']
{'class0': 93.81656023949998, 'class1': 87.68150096878921}
['splitting-clusters', 3, 'periodic', 'WEOB2']
{'class0': 93.92259938870674, 'class1': 87.80824697730587}
['splitting-clusters', 3, 'periodic', 'UOB']
{'class0': 94.14063340621257, 'class1': 87.7360692476752}
['splitting-clusters', 3, 'periodic', 'RebalanceStream']
{'class0': 94.4802209596999, 'class1': 87.7316403324725}
['splitting-clusters', 4, 'incremental', 'improved_OOB']
{'class0': 91.80526394744801, 'class1': 89.72831673035941}
['splitting-clusters', 4, 'incremental', 'WEOB1']
{'class0': 91.72142852182427, 'class1': 89.68173555989253}
['splitting-clusters', 4, 'incremental', 'CSMOTE']
{'class0': 91.04471949542767, 'class1': 88.2988251680489}
['splitting-clusters', 4, 'incremental', 'OOB']
{'class0': 91.4658667

In [5]:
def stats_of(_dict ):
    values_0 = []
    values_1 = []
    for alg, values in _dict.items():
        values_0.append(values["class0"])
        values_1.append(values["class1"])
    return np.mean(values_0),np.mean(values_1), np.max(values_0), np.max(values_1), np.min(values_0), np.min(values_1)


In [18]:
with open('F1XF1.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["drift","unbalance","type","alg","F1_0", "F1_1","F1_0%", "F1_1%","F1_0_Mean", "F1_1_Mean"])
    for drift, dicts in final_dict.items():
        for unbalance, dtypes in dicts.items():
            for dtype, algs in dtypes.items():
                mean0, mean1,max0,max1,min0,min1 = stats_of(algs)
                for alg, values in algs.items():
                    print([drift,unbalance,dtype,alg])
                    print(values)
                    mean_0 = (values["class0"]-mean0)
                    mean_1 = (values["class1"]-mean1)
                    perc_0 = (values["class0"]/mean0)
                    perc_1 = (values["class1"]/mean1)
                    writer.writerow([drift,unbalance,dtype,alg,values["class0"],values["class1"],perc_0,perc_1,mean_0,mean_1])

['sea_P(y)', 1, 'sudden', 'improved_OOB']
{'class0': 99.04130893811295, 'class1': 98.10819729063016}
['sea_P(y)', 1, 'sudden', 'WEOB1']
{'class0': 98.97942226181465, 'class1': 97.83921843449522}
['sea_P(y)', 1, 'sudden', 'OOB']
{'class0': 98.98612670033285, 'class1': 98.03087014819405}
['sea_P(y)', 1, 'sudden', 'ESOS_ELM']
{'class0': 89.30415619559611, 'class1': 73.42351874845741}
['sea_P(y)', 1, 'sudden', 'improved_UOB']
{'class0': 98.81573147725297, 'class1': 96.9240271514656}
['sea_P(y)', 1, 'sudden', 'WEOB2']
{'class0': 98.6244127997512, 'class1': 96.19870135316566}
['sea_P(y)', 1, 'sudden', 'UOB']
{'class0': 98.98612670033285, 'class1': 98.03087014819405}
['sea_P(y)', 1, 'sudden', 'CSMOTE']
{'class0': 98.74740154470427, 'class1': 97.13040882292873}
['sea_P(y)', 1, 'sudden', 'RebalanceStream']
{'class0': 98.4522316004296, 'class1': 93.46004270685152}
['sea_P(y)', 1, 'incremental', 'improved_OOB']
{'class0': 99.07781073529245, 'class1': 98.36413751297053}
['sea_P(y)', 1, 'incrementa

{'class0': 92.20811753330409, 'class1': 90.68984188672906}
['jitter', 4, 'sudden', 'WEOB1']
{'class0': 91.86481301790903, 'class1': 90.41684038885866}
['jitter', 4, 'sudden', 'CSMOTE']
{'class0': 86.67033936227622, 'class1': 84.5944072258466}
['jitter', 4, 'sudden', 'OOB']
{'class0': 92.27438468331613, 'class1': 90.3556771540433}
['jitter', 4, 'sudden', 'ESOS_ELM']
{'class0': 87.52282666632702, 'class1': 72.41214736084105}
['jitter', 4, 'sudden', 'improved_UOB']
{'class0': 90.91995217988074, 'class1': 89.47683649347536}
['jitter', 4, 'sudden', 'WEOB2']
{'class0': 85.76663724631123, 'class1': 83.90378926015373}
['jitter', 4, 'sudden', 'UOB']
{'class0': 92.27438468331613, 'class1': 90.3556771540433}
['jitter', 4, 'sudden', 'RebalanceStream']
{'class0': 92.07709089031471, 'class1': 89.94813795238338}
['jitter', 4, 'periodic', 'improved_OOB']
{'class0': 88.9193216579731, 'class1': 86.52926477018327}
['jitter', 4, 'periodic', 'WEOB1']
{'class0': 88.22915893607203, 'class1': 85.9165390901670

{'class0': 80.09994480684756, 'class1': 65.6747922903127}
['appearing-clusters', 4, 'sudden', 'improved_UOB']
{'class0': 92.21356918394689, 'class1': 90.07738588081112}
['appearing-clusters', 4, 'sudden', 'WEOB2']
{'class0': 92.18101294332027, 'class1': 89.92971110108759}
['appearing-clusters', 4, 'sudden', 'UOB']
{'class0': 92.99097083707241, 'class1': 90.68696524872766}
['appearing-clusters', 4, 'sudden', 'RebalanceStream']
{'class0': 92.22620004599506, 'class1': 89.73059841739703}
['appearing-clusters', 4, 'periodic', 'improved_OOB']
{'class0': 95.85180476585606, 'class1': 94.34742143452722}
['appearing-clusters', 4, 'periodic', 'WEOB1']
{'class0': 95.79016930173773, 'class1': 94.26818682019027}
['appearing-clusters', 4, 'periodic', 'CSMOTE']
{'class0': 95.94974915732197, 'class1': 94.43383968252552}
['appearing-clusters', 4, 'periodic', 'OOB']
{'class0': 96.08893665163723, 'class1': 94.60659109090395}
['appearing-clusters', 4, 'periodic', 'ESOS_ELM']
{'class0': 83.57271049272886, '

['minority-share', 3, 'incremental', 'WEOB1']
{'class0': 88.38980161301768, 'class1': 77.54917065103893}
['minority-share', 3, 'incremental', 'CSMOTE']
{'class0': 88.5094269517729, 'class1': 76.42199113054701}
['minority-share', 3, 'incremental', 'OOB']
{'class0': 88.32048306124025, 'class1': 77.11542852358465}
['minority-share', 3, 'incremental', 'ESOS_ELM']
{'class0': 82.73075056386317, 'class1': 56.66241986657089}
['minority-share', 3, 'incremental', 'improved_UOB']
{'class0': 87.40789081014869, 'class1': 76.50282608690733}
['minority-share', 3, 'incremental', 'WEOB2']
{'class0': 86.6577500906551, 'class1': 75.45958247551356}
['minority-share', 3, 'incremental', 'UOB']
{'class0': 85.62973124877706, 'class1': 74.93761410271017}
['minority-share', 3, 'incremental', 'RebalanceStream']
{'class0': 87.6521033764467, 'class1': 65.66899371380393}
['minority-share', 3, 'sudden', 'improved_OOB']
{'class0': 89.19439191720996, 'class1': 81.07953811220446}
['minority-share', 3, 'sudden', 'WEOB1'

['clusters-movement', 3, 'periodic', 'improved_OOB']
{'class0': 87.82435710208054, 'class1': 78.25349694830729}
['clusters-movement', 3, 'periodic', 'WEOB1']
{'class0': 86.94891028201482, 'class1': 77.21349813651258}
['clusters-movement', 3, 'periodic', 'CSMOTE']
{'class0': 86.23429988486882, 'class1': 75.95331715962831}
['clusters-movement', 3, 'periodic', 'OOB']
{'class0': 87.16696945046161, 'class1': 78.31953011165413}
['clusters-movement', 3, 'periodic', 'ESOS_ELM']
{'class0': 81.80545855186537, 'class1': 58.21472458065332}
['clusters-movement', 3, 'periodic', 'improved_UOB']
{'class0': 84.95385778243062, 'class1': 74.7375756490057}
['clusters-movement', 3, 'periodic', 'WEOB2']
{'class0': 84.46282970231775, 'class1': 74.17219428930785}
['clusters-movement', 3, 'periodic', 'UOB']
{'class0': 82.53072314463382, 'class1': 72.87516609874498}
['clusters-movement', 3, 'periodic', 'RebalanceStream']
{'class0': 86.78785323781607, 'class1': 71.16668525620797}
['clusters-movement', 4, 'increm

# REALS

In [6]:
real_dict = {}
real_dict["KDDCup"] = {}
real_dict["PAKDD"] = {}
real_dict["Elec"] = {}

for i in real_dict.keys():
    path = f"Results_union/Real/{i}"
    for alg in os.listdir(path):
        class0 = []
        if "DS_Store" in alg or "ipynb" in alg:
            continue
        class1 = []
        if alg not in real_dict[i]:
            real_dict[i][alg] = {}
        
            
        for exp in os.listdir(path + "/" + alg): 
            if "err" not in exp or "DS_Store" in exp:
                continue
                
            data = import_csv(path + "/" + alg + "/"+ exp)
            daily0 = []
            daily1 = []
            start = False

            for data_row in data:
                if data_row[1] == "learning evaluation instances":
                    start = True
                    continue
                if not start:
                    continue

                try:
                    daily0.append(float(data_row[9]))
                except:
                    if data_row[9] == "?":
                        daily0.append(0.0)
                    else:
                        print("ZERO")
                        print(data_row)
                try:
                    daily1.append(float(data_row[10]))
                except:
                    if data_row[10] == "?":
                        daily1.append(0.0)
                    else:
                        print("ONE")
                        print(data_row)
            try:
                if len(daily0)>0 and len(daily1)>0:
                    class0.append(np.mean(daily0))
                    class1.append(np.mean(daily1))
            except:
                print(f"error with {result}")
        
        real_dict[i][alg]["class0"] = np.mean(class0)
        real_dict[i][alg]["class1"] = np.mean(class1)
        
                
                

In [7]:
real_dict

{'KDDCup': {'improved_OOB': {'class0': 97.92131092513542,
   'class1': 91.26893835964778},
  'WEOB1': {'class0': 95.63253800339828, 'class1': 91.14788385158384},
  'OOB': {'class0': 97.71239503010152, 'class1': 92.83696092715991},
  'ESOS_ELM': {'class0': 83.12720197527338, 'class1': 0.0},
  'improved_UOB': {'class0': 92.06415723668195, 'class1': 89.58295496510378},
  'WEOB2': {'class0': 92.4518633128902, 'class1': 81.49641525620021},
  'UOB': {'class0': 95.05175395125022, 'class1': 94.35237962976271},
  'OSHAT': {'class0': 97.32720428082052, 'class1': 97.32066329951691},
  'RebalanceStream': {'class0': 97.5773771472323,
   'class1': 93.58443221469317}},
 'PAKDD': {'improved_OOB': {'class0': 79.53010509624917,
   'class1': 34.207625017294596},
  'WEOB1': {'class0': 71.78711156698657, 'class1': 36.89125649360513},
  'OOB': {'class0': 70.41285339081057, 'class1': 36.76346877589926},
  'ESOS_ELM': {'class0': 69.69740851952237, 'class1': 25.865104540765753},
  'improved_UOB': {'class0': 63

In [8]:
with open('F1XF1_real.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["ds","alg","F1_0", "F1_1"])
    for ds, dicts in real_dict.items():
        mean0, mean1,max0,max1,min0,min1 = stats_of(dicts)
        for alg, values in dicts.items():
            mean_0 = (values["class0"]-mean0)
            mean_1 = (values["class1"]-mean1)
            perc_0 = (values["class0"]/mean0)
            perc_1 = (values["class1"]/mean1)
            if alg == "OSHAT":
                alg = "CSMOTE"
            writer.writerow([ds,alg,values["class0"],values["class1"],perc_0,perc_1,mean_0,mean_1])
